In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np
from simuq.qsystem import QSystem
from simuq.environment import qubit
from scipy.constants import h
from backends.qiskit_pulse_ibm import get_n_link
from simuq.solver import generate_as
from aais.ibm import get_mach
from qiskit_ibm_provider import IBMProvider
from qiskit import QuantumCircuit
provider = IBMProvider()
backend = provider.get_backend("ibmq_guadalupe")
mach=get_mach(backend)

from qiskit_ibm_provider import IBMProvider
provider = IBMProvider(instance="open",hub="ibm-q",group="open",project="main")
from backends.qiskit_pulse_ibm import transpile
from qiskit import transpile as transpile_qiskit
simulator=provider.get_backend("ibmq_qasm_simulator")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
circ_list=[]
chain=True
n = 6
for T in [0.2,0.4,0.6,0.8,1]:
    qs = QSystem()
    qubits = [qubit(qs) for i in range(n)]
    H0 = 0
    if chain:
        for i in range(n-1):
            H0 += qubits[i].X * qubits[(i+1)%n].X
    else:
        for i in range(n):
            H0 += qubits[i].X * qubits[(i+1)%n].X

    for i in range(n):
        H0 += - qubits[i].Z

    qs.add_evolution(H0, T)
    circ = transpile(backend,*generate_as(qs, mach,4),False)
    active_qubits=[0,1,2,3,5,8]
    h_circ=QuantumCircuit(16)

    for idx in active_qubits:
        circ.h(idx)
        h_circ.h(idx)
    new_circ=h_circ.compose(circ)
    new_circ.measure_active()
    new_circ=transpile_qiskit(new_circ,backend)
    circ_list.append(new_circ)
job = backend.run(circ_list)
print(job.job_id())
count=job.result().get_counts()
# print(count["000000"]/4000)    

First round time:  0.036783695220947266
Second round time:  0.00625920295715332


First round time:  0.03567624092102051
Second round time:  0.00628352165222168
First round time:  0.03539538383483887
Second round time:  0.005750179290771484
First round time:  0.037680864334106445
Second round time:  0.006081581115722656
First round time:  0.03527998924255371
Second round time:  0.0057086944580078125
cifp9iu4c9s1cd74g93g


In [7]:
from qiskit import schedule
from backends.qiskit_pulse_ibm import transpile

chain=True
n = 6
T=1
qs = QSystem()
qubits = [qubit(qs) for i in range(n)]
H0 = 0
if chain:
    for i in range(n-1):
        H0 += qubits[i].X * qubits[(i+1)%n].X
else:
    for i in range(n):
        H0 += qubits[i].X * qubits[(i+1)%n].X

for i in range(n):
    H0 += - qubits[i].Z

qs.add_evolution(H0, T)
circ = transpile(backend,*generate_as(qs, mach,4),use_pulse=True)
active_qubits=[0,1,2,3,5,8]
h_circ=QuantumCircuit(16)

for idx in active_qubits:
    circ.h(idx)
    h_circ.h(idx)
new_circ=h_circ.compose(circ)
new_circ=transpile_qiskit(new_circ,backend)
schedule(new_circ,backend).duration

First round time:  0.14225506782531738
Second round time:  0.024569988250732422


10368